In [17]:
import pulp as p


def balance_linea(tareas, estaciones, tiempos, tiempo_ciclo, num_precedencias,precedencias):
    
    #Definir modelo
    balance=p.LpProblem("balance de linea", p.LpMinimize)
    
    #Crear variables de decisión
    x=p.LpVariable.dicts("x", (range(tareas), range(estaciones)), cat='Binary') #variables binarias
    
    #Definir funciòn objetivo
    balance += p.lpSum((2**j)*x[i][j] for i in range (tareas) for j in range(estaciones))

    #Restricción: Asignación
    for i in range(tareas):
        balance += p.lpSum(x[i][j] for j in range(estaciones)) == 1

    #Restricción: Capacidad total
    for j in range(estaciones):
        balance += p.lpSum(tiempos[i]*x[i][j] for i in range (tareas))<=tiempo_ciclo

    #Restricción: Precedencias
    for k in range (num_precedencias+1):
        predecesora,seguidora = precedencias[k]
        for js in range (estaciones):
            balance += p.lpSum(x[seguidora][js]-(x[predecesora][j] for j in range (js+1)))<=0

    #Resolver balance
    balance.solve()

    #Resultados:
    if balance.status == p.LpStatusOptimal:
        tiempos_estaciones = []
        for j in range(estaciones): 
            tiempo_estación = sum(tiempos[i] * x[i][j].value() for i in range(tareas))
            tiempos_estaciones.append(tiempo_estación)
            utilización = (tiempo_estación / tiempo_ciclo) * 100
            print(f'Estación {j + 1}: Tiempo total = {tiempo_estación:.2f}, Porcentaje de utilización = {utilización:.2f}%')

        # Imprimir asignación de tareas a estaciones
        for i in range(tareas):
            for j in range(estaciones):
                if x[i][j].varValue == 1:
                    print(f"Tarea {i + 1} asignada a la Estación {j + 1}")
    else:
        print("No se encontró ninguna solución factible.")
    for i in range(tareas):
        for j in range(estaciones):
            print(f"x[{i}][{j}] = {x[i][j].varValue}")
        
# Solicitar entradas al usuario
tareas = int(input("Ingrese el número de tareas: "))
estaciones = int(input("Ingrese el número de estaciones: "))
tiempos = []
for i in range(tareas):
    tiempo = float(input(f"Ingrese el tiempo de procesamiento para la tarea {i + 1}: "))
    tiempos.append(tiempo)
tiempo_ciclo = float(input("Ingrese el tiempo máximo por ciclo: "))
num_precedencias=int(input("Ingrese el número de relaciones de precedencia: "))

# Precedencia
precedencias=[]
for i in range (tareas):
    num=int(input(f"¿Cuantas precedencias tiene la tarea {i+1}?"))
    for k in range (num):
        precedencia=int(input(f"precedencia {k+1} de la tarea {i+1}"))-1
        precedencias.append((precedencia,i))
print(precedencias)        
balance_linea(tareas, estaciones, tiempos, tiempo_ciclo, num_precedencias, precedencias)

[(0, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 6), (3, 6), (6, 7), (4, 8), (5, 9), (8, 9), (7, 10), (9, 10), (10, 11)]
Estación 1: Tiempo total = 10.20, Porcentaje de utilización = 80.95%
Estación 2: Tiempo total = 11.40, Porcentaje de utilización = 90.48%
Estación 3: Tiempo total = 12.40, Porcentaje de utilización = 98.41%
Estación 4: Tiempo total = 12.40, Porcentaje de utilización = 98.41%
Estación 5: Tiempo total = 11.80, Porcentaje de utilización = 93.65%
Estación 6: Tiempo total = 7.10, Porcentaje de utilización = 56.35%
Tarea 1 asignada a la Estación 1
Tarea 2 asignada a la Estación 2
Tarea 3 asignada a la Estación 2
Tarea 4 asignada a la Estación 4
Tarea 5 asignada a la Estación 3
Tarea 6 asignada a la Estación 3
Tarea 7 asignada a la Estación 4
Tarea 8 asignada a la Estación 5
Tarea 9 asignada a la Estación 3
Tarea 10 asignada a la Estación 4
Tarea 11 asignada a la Estación 5
Tarea 12 asignada a la Estación 6
x[0][0] = 1.0
x[0][1] = 0.0
x[0][2] = 0.0
x[0][3] = 0.0
x[0][4] = 0.0
x